In [1]:
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
url = 'https://www.metacritic.com/browse/games/score/metascore/year/all/filtered?year_selected=year_str&distribution=&sort=desc&view=detailed'

#creating a list of URLs for each year
year_url = lambda x:url.replace("year_str", str(x))
mc_years = [year_url(i) for i in range(2005,2022)]

mc_years = []
mc_year_list = {'2005':[],'2006':[], '2007':[] ,'2008':[] ,'2009':[], '2010':[], '2011':[], '2012':[], '2013':[], '2014':[], '2015':[], '2016':[], '2017':[], '2018':[], '2019':[], '2020':[], '2021':[]}

for i in range (2005, 2022):
    final_url = 'https://www.metacritic.com/browse/games/score/metascore/year/all/filtered?year_selected=' + "{0}".format(i) + '&distribution=&sort=desc&view=detailed'
    mc_years.append(final_url)

In [3]:
s=Service(ChromeDriverManager().install())

#toggle needed options
options = Options()
options.headless = True #open browser unseen

#create driver
driver = webdriver.Chrome(service=s, options=options)
#---------------------------------------------------------------------------
    
for years, details in zip(mc_years, mc_year_list):
    driver.get(years)
    ps = driver.page_source
    soup = BeautifulSoup(ps, 'html.parser')
    page_numbers = int(soup.find("li", class_ = "page last_page").find("a").string)
    for i in range (0, page_numbers):
        driver.get(years + '&page=' + "{0}".format(i))
        ps = driver.page_source
        soup = BeautifulSoup(ps, 'html.parser')
        mc_year_list[details].append(soup.find_all(class_ = "clamp-summary-wrap"))

driver.quit()



====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [C:\Users\Администратор\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


In [4]:
mc_name = []
mc_score = []

for year in mc_year_list:
    for i in range(0, len(mc_year_list[year])):
        for j in range (0, len(mc_year_list[year][i])):
            mc_name.append(mc_year_list[year][i][j].find("a", class_ = "title").string)
            check = mc_year_list[year][i][j].find("div", class_ = "metascore_w large game positive")
            if (check == None):
                check = mc_year_list[year][i][j].find("div", class_ = "metascore_w large game negative")
                if (check == None):
                    check = mc_year_list[year][i][j].find("div", class_ = "metascore_w large game mixed")
                    if(check == None):
                        mc_score.append(None)
                    else:
                        mc_score.append(check.string)
                else:
                    mc_score.append(check.string)
            else:
                mc_score.append(check.string)

In [5]:
metacritic_df = pd.DataFrame({'name':mc_name, 'score':mc_score})
metacritic_df.to_csv("metacritic.csv")

In [22]:
# get API and keep the important columns. 

#build first dataframe
url = "https://steamspy.com/api.php?request=all"
res = requests.get(url)
data_json = res.json()

steam_spy = pd.DataFrame.from_dict(data_json, orient='index')

## Return format for an app: ##

  * appid - Steam Application ID. If it's 999999, then data for this application is hidden on developer's request, sorry.
  * name - game's name
  * developer - comma separated list of the developers of the game
  * publisher - comma separated list of the publishers of the game
  * score_rank - score rank of the game based on user reviews
  * owners - owners of this application on Steam as a range.
  * average_forever - average playtime since March 2009. In minutes.
  * average_2weeks - average playtime in the last two weeks. In minutes.
  * median_forever - median playtime since March 2009. In minutes.
  * median_2weeks - median playtime in the last two weeks. In minutes.
  * ccu - peak CCU yesterday.
  * price - current US price in cents.
  * initialprice - original US price in cents.
  * discount - current discount in percents.
  * tags - game's tags with votes in JSON array.
  * languages - list of supported languages.
  * genre - list of genres.

In [23]:
steam_spy.sort_values('positive', axis=0, ascending=False)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
730,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,,5718191,761211,0,"50,000,000 .. 100,000,000",28846,870,6493,292,0,0,0,809427
570,570,Dota 2,Valve,Valve,,1467658,297030,0,"100,000,000 .. 200,000,000",38219,1622,964,797,0,0,0,631184
271590,271590,Grand Theft Auto V,Rockstar North,Rockstar Games,,1153983,208800,0,"20,000,000 .. 50,000,000",12460,935,6441,168,1480,1480,0,117793
578080,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",,1146769,892200,0,"50,000,000 .. 100,000,000",21155,853,6620,229,0,0,0,353383
105600,105600,Terraria,Re-Logic,Re-Logic,,951689,20646,0,"20,000,000 .. 50,000,000",6126,463,1739,164,499,999,50,44007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1887990,1887990,Deep Night Runner,DODRECK,kazakovstudios,,11,2,0,"1,000,000 .. 2,000,000",0,0,0,0,874,2499,65,1
1353950,1353950,Ring of Fire: Prologue,"de Fault, Chard",Far Few Giants,,9,1,0,"1,000,000 .. 2,000,000",0,0,0,0,0,0,0,0
21110,21110,F.E.A.R.,"Monolith Productions, Inc., Timegate","Warner Bros. Games, Warner Bros. Interactive E...",,0,0,0,"1,000,000 .. 2,000,000",312,0,344,0,824,5499,85,12
21120,21120,F.E.A.R.,"Monolith Productions, Inc., Timegate","Warner Bros. Games, Warner Bros. Interactive E...",,0,0,0,"1,000,000 .. 2,000,000",441,0,521,0,824,5499,85,14


In [33]:
#concat other dataframes
for i in range(1,10):
    URL = url + "&page=" + "{0}".format(i)
    res = requests.get(URL)
    data_json = res.json()
    new_pd = pd.DataFrame.from_dict(data_json, orient='index')
    steam_spy= pd.concat([steam_spy, new_pd], axis = 0)
    


  Returns all games with owners data sorted by owners. Returns 1,000 entries per page.
  * page - page number for the list (starts at 0)

When we concatenate DataFrames, we need to specify the axis. axis=0 tells pandas to stack the second DataFrame UNDER the first one. It will automatically detect whether the column names are the same and will stack accordingly. axis=1 will stack the columns in the second DataFrame to the RIGHT of the first DataFrame.

In [34]:
steam_spy.drop(columns=[ 'appid', 'score_rank', 'ccu', 'average_2weeks', 'median_2weeks', 'initialprice', 'discount', 'userscore'], inplace=True)
steam_spy = steam_spy.sort_values('positive', axis=0, ascending=False)
steam_spy.drop_duplicates(keep='first', inplace=True)
steam_spy = steam_spy[steam_spy["name"].str.contains("Edition")==False]

droping unnecessay columns, duplicates, removing spacial editions

In [35]:
steam_spy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9639 entries, 730 to 8640
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             9639 non-null   object
 1   developer        9639 non-null   object
 2   publisher        9639 non-null   object
 3   positive         9639 non-null   int64 
 4   negative         9639 non-null   int64 
 5   owners           9639 non-null   object
 6   average_forever  9639 non-null   int64 
 7   median_forever   9639 non-null   int64 
 8   price            9639 non-null   object
dtypes: int64(4), object(5)
memory usage: 753.0+ KB


In [36]:
steam_spy.info(verbose = False)

<class 'pandas.core.frame.DataFrame'>
Index: 9639 entries, 730 to 8640
Columns: 9 entries, name to price
dtypes: int64(4), object(5)
memory usage: 753.0+ KB


In [37]:
steam_spy.to_csv("steam_spy_final.csv")

In [38]:
steam_spy_final = pd.read_csv("steam_spy_final.csv")

In [39]:
steam_spy_final

,Unnamed: 0,name,developer,publisher,positive,negative,owners,average_forever,median_forever,price
0,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,5718191,761211,"50,000,000 .. 100,000,000",28846,6493,0
1,570,Dota 2,Valve,Valve,1467658,297030,"100,000,000 .. 200,000,000",38219,964,0
2,271590,Grand Theft Auto V,Rockstar North,Rockstar Games,1153983,208800,"20,000,000 .. 50,000,000",12460,6441,1480
3,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",1146769,892200,"50,000,000 .. 100,000,000",21155,6620,0
4,105600,Terraria,Re-Logic,Re-Logic,951689,20646,"20,000,000 .. 50,000,000",6126,1739,499
...,...,...,...,...,...,...,...,...,...,...
9634,1313330,HEDE Game Engine,Hede Games,Hede Games,0,7,"50,000 .. 100,000",0,0,99
9635,1599220,League Space,EURO GAMES STUDIO,EURO GAMES STUDIO (ESP),0,1,"50,000 .. 100,000",0,0,1199
9636,1434470,RagDoll MadDoll,Team Booky,Dystopian Edge Publishing,0,1,"50,000 .. 100,000",0,0,124
9637,1627280,Operation: VICUS,"Ilja Soutchilin, Tom Berger","Ilja Soutchilin, Tom Berger",0,2,"50,000 .. 100,000",0,0,799


In [40]:
steam_spy_final.drop(columns=["Unnamed: 0"], inplace=True)
steam_spy_final.reset_index(drop=True)
steam_spy_final

,name,developer,publisher,positive,negative,owners,average_forever,median_forever,price
0,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,5718191,761211,"50,000,000 .. 100,000,000",28846,6493,0
1,Dota 2,Valve,Valve,1467658,297030,"100,000,000 .. 200,000,000",38219,964,0
2,Grand Theft Auto V,Rockstar North,Rockstar Games,1153983,208800,"20,000,000 .. 50,000,000",12460,6441,1480
3,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",1146769,892200,"50,000,000 .. 100,000,000",21155,6620,0
4,Terraria,Re-Logic,Re-Logic,951689,20646,"20,000,000 .. 50,000,000",6126,1739,499
...,...,...,...,...,...,...,...,...,...
9634,HEDE Game Engine,Hede Games,Hede Games,0,7,"50,000 .. 100,000",0,0,99
9635,League Space,EURO GAMES STUDIO,EURO GAMES STUDIO (ESP),0,1,"50,000 .. 100,000",0,0,1199
9636,RagDoll MadDoll,Team Booky,Dystopian Edge Publishing,0,1,"50,000 .. 100,000",0,0,124
9637,Operation: VICUS,"Ilja Soutchilin, Tom Berger","Ilja Soutchilin, Tom Berger",0,2,"50,000 .. 100,000",0,0,799


In [41]:
steam_spy_final.to_csv("steam_spy_final.csv")